In [1]:
!pip install -q -U watermark

In [40]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.4/838.4 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66

In [2]:
!pip uninstall -y transformers

Found existing installation: transformers 4.48.0
Uninstalling transformers-4.48.0:
  Successfully uninstalled transformers-4.48.0


In [3]:
!pip install transformers

  Using cached transformers-4.48.0-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.48.0-py3-none-any.whl (9.7 MB)


In [4]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,transformers

Python implementation: CPython
Python version       : 3.11.11
IPython version      : 7.34.0

numpy       : 1.26.4
pandas      : 2.2.2
torch       : 2.5.1+cu118
transformers: 4.48.0



In [5]:
import transformers
from transformers import BertModel, AutoTokenizer, BertTokenizer, PreTrainedTokenizerFast, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F

import torch
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss



##Trainer от HuggingFace

In [6]:
!pip install datasets

In [7]:
from datasets import load_dataset

imdb_dataset = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
imdb_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
imdb_dataset["train"]["text"][:10]

['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, e

In [8]:
num_cls = len(set(imdb_dataset["train"]["label"]))

Сократим количество примеров, чтобы процесс обучения был быстрее.

In [9]:
train_texts = imdb_dataset["train"]["text"][:2000]
train_labels = imdb_dataset["train"]["label"][:2000]
test_texts = imdb_dataset["test"]["text"][:1000]
test_labels = imdb_dataset["test"]["label"][:1000]

In [10]:
del imdb_dataset

Выделим часть обучающих данных для оценки:

In [11]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2)

Токенизируем данные

In [12]:
from transformers import AutoTokenizer, AutoModel
PRE_TRAINED_MODEL_NAME = "albert/albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [13]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

Создадим класс Dataset, который на каждой итерации будет возвращать токенизированный отзыв и класс:

In [14]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)

Опишем функцию для подсчета метрик, которые хотим увидеть при оценке модели (по умолчанию будет вычисляться только loss):

In [15]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions


    preds = torch.tensor(preds).cpu()
    labels = torch.tensor(labels).cpu()

    predictions = torch.argmax(preds, axis=-1)

    predictions = predictions.numpy()
    labels = labels.numpy()

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

Зададим параметры обучения с помощью TrainingArguments:

In [16]:
RANDOM_SEED = 1
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



In [17]:
device

device(type='cuda', index=0)

In [19]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10
)

Инициализируем модель и запустим fine-tune ее с помощью модуля Trainer. В данном модуле будет автоматически происходить итерация по эпохам, батчам, рассчет функции потерь и градиентов, ничего этого больше руками прописывать не нужно!

In [19]:
class SentimentClassifier(nn.Module):

  def __init__(self, n_classes):
    super().__init__()
    self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask, labels=None):
    outputs = self.bert(
        input_ids=input_ids,
        attention_mask=attention_mask,
        return_dict=False
    )
    pooled_output = outputs[1] #только выход с пуллер-слоя
    logits = self.out(self.drop(pooled_output))

    loss = None
    if labels is not None:
        loss_fn = CrossEntropyLoss()
        loss = loss_fn(logits, labels)

    return {"loss": loss, "logits": logits}

In [ ]:
model = SentimentClassifier(num_cls)


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

Step,Training Loss
10,0.684100
20,0.518900
30,0.360800
40,0.178900
50,0.068300
60,0.016300
70,0.003200
80,0.001400
90,0.000800
100,0.000600


TrainOutput(global_step=400, training_loss=0.04590843525224045, metrics={'train_runtime': 319.4827, 'train_samples_per_second': 10.016, 'train_steps_per_second': 1.252, 'total_flos': 0.0, 'train_loss': 0.04590843525224045, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 2.1158759409445338e-05,
 'eval_accuracy': 1.0,
 'eval_f1': 0.0,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 13.3608,
 'eval_samples_per_second': 29.938,
 'eval_steps_per_second': 1.871,
 'epoch': 2.0}

Оценим модель на тестовых данных

In [ ]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'test_loss': 1.9349943613633513e-05,
 'test_accuracy': 1.0,
 'test_f1': 0.0,
 'test_precision': 0.0,
 'test_recall': 0.0,
 'test_runtime': 34.7731,
 'test_samples_per_second': 28.758,
 'test_steps_per_second': 1.812,
 'epoch': 2.0}

Выход с пуллер-слоя + эмбеддинг cls-токена с последнего слоя:

In [20]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes):
        super().__init__()
        self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.3)
        self.out = nn.Linear(self.bert.config.hidden_size * 2, n_classes)  #увеличила входной размер, т.к. будем конкатенировать выходы с пуллер-слоя и с последнего

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False
        )


        pooled_output = outputs[1]  #выход с пуллер слоя


        cls_embedding = outputs[0][:, 0, :]  #cls-токены (как я поняла, можно еще было сделать return_dict=True, cls_embedding = outputs['last_hidden_state'][:, 0, :])


        combined_output = torch.cat((pooled_output, cls_embedding), dim=1)  #конкатенируем, получаем матрицу [batch_size, hidden_size*2]


        logits = self.out(self.drop(combined_output))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

In [21]:
model2 = SentimentClassifier(num_cls)
model2.to(device)

SentimentClassifier(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True

In [22]:
trainer = Trainer(
    model=model2,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss
10,0.764100
20,0.329300
30,0.080500
40,0.007100
50,0.000600
60,0.000100
70,0.000100
80,0.000100
90,0.000000
100,0.000000


TrainOutput(global_step=400, training_loss=0.02955936964959619, metrics={'train_runtime': 467.6343, 'train_samples_per_second': 6.843, 'train_steps_per_second': 0.855, 'total_flos': 0.0, 'train_loss': 0.02955936964959619, 'epoch': 2.0})

In [23]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'test_loss': 2.6003908715210855e-06,
 'test_accuracy': 1.0,
 'test_f1': 0.0,
 'test_precision': 0.0,
 'test_recall': 0.0,
 'test_runtime': 32.0886,
 'test_samples_per_second': 31.164,
 'test_steps_per_second': 1.963,
 'epoch': 2.0}

cls-токены с нескольких слоев:  
В этот раз решила не конкатенировать векторы, а брать средний из всех cls, чтобы он не поличился слишком огромный

In [ ]:
class SentimentClassifier(nn.Module):
    def __init__(self, n_classes, selected_layers=None, dropout=0.3):
        super().__init__()
        self.bert = AutoModel.from_pretrained(PRE_TRAINED_MODEL_NAME, output_hidden_states=True)
        self.selected_layers = selected_layers if selected_layers else [-1]   #если модель не получит кол-во слоев, с которых хотим брать cls, она возьмет их только с последнего
        self.hidden_size = self.bert.config.hidden_size
        self.drop = nn.Dropout(dropout)
        self.out = nn.Linear(self.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True
        )
        hidden_states = outputs.hidden_states  #список скрытых состояний

        cls_tokens = [hidden_states[i][:, 0, :] for i in self.selected_layers]  #cls из скрытых состояний

        averaged_cls = torch.mean(torch.stack(cls_tokens), dim=0) #усредняем

        logits = self.out(self.drop(averaged_cls))

        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()
            loss = loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

In [ ]:
LAYERS = [0, 4, 7, 11] #сразу со всех решила не собирать

In [ ]:
model3 = SentimentClassifier(num_cls, LAYERS)

In [ ]:
trainer = Trainer(
    model=model3,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

Step,Training Loss
10,0.642600
20,0.551200
30,0.360600
40,0.152900
50,0.039700
60,0.004500
70,0.001000
80,0.000400
90,0.000200
100,0.000200


TrainOutput(global_step=400, training_loss=0.04386173108337971, metrics={'train_runtime': 303.4685, 'train_samples_per_second': 10.545, 'train_steps_per_second': 1.318, 'total_flos': 0.0, 'train_loss': 0.04386173108337971, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 9.435364518139977e-06,
 'eval_accuracy': 1.0,
 'eval_f1': 0.0,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 13.3272,
 'eval_samples_per_second': 30.014,
 'eval_steps_per_second': 1.876,
 'epoch': 2.0}

In [ ]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'test_loss': 1.0041400855698157e-05,
 'test_accuracy': 1.0,
 'test_f1': 0.0,
 'test_precision': 0.0,
 'test_recall': 0.0,
 'test_runtime': 34.7644,
 'test_samples_per_second': 28.765,
 'test_steps_per_second': 1.812,
 'epoch': 2.0}

In [ ]:
LAYERS = [0, 4, 7, 11] #попробуем все слои

In [ ]:
model3 = SentimentClassifier(num_cls, LAYERS)

In [ ]:
trainer = Trainer(
    model=model3,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

Step,Training Loss
10,0.798300
20,0.665100
30,0.441400
40,0.196500
50,0.045000
60,0.005400
70,0.001200
80,0.000600
90,0.000300
100,0.000200


TrainOutput(global_step=400, training_loss=0.053894702858597154, metrics={'train_runtime': 302.7403, 'train_samples_per_second': 10.57, 'train_steps_per_second': 1.321, 'total_flos': 0.0, 'train_loss': 0.053894702858597154, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 1.0115742043126374e-05,
 'eval_accuracy': 1.0,
 'eval_f1': 0.0,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 13.4309,
 'eval_samples_per_second': 29.782,
 'eval_steps_per_second': 1.861,
 'epoch': 2.0}

In [ ]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'test_loss': 1.1470053323137108e-05,
 'test_accuracy': 1.0,
 'test_f1': 0.0,
 'test_precision': 0.0,
 'test_recall': 0.0,
 'test_runtime': 33.767,
 'test_samples_per_second': 29.615,
 'test_steps_per_second': 1.866,
 'epoch': 2.0}

Вроде бы, просто оценки на валидации и тесте поближе друг к другу стали.

Применение готовой модели:

In [ ]:
from transformers import AutoModelForSequenceClassification
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model4 = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
trainer = Trainer(
    model=model4,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics = compute_metrics    # metrics to evaluate
)

trainer.train()

Step,Training Loss
10,2.363300
20,0.938200
30,0.076800
40,0.020700
50,0.009900
60,0.005400
70,0.003800
80,0.002500
90,0.002100
100,0.001600


Step,Training Loss
10,2.363300
20,0.938200
30,0.076800
40,0.020700
50,0.009900
60,0.005400
70,0.003800
80,0.002500
90,0.002100
100,0.001600


TrainOutput(global_step=400, training_loss=0.08587084488681285, metrics={'train_runtime': 327.3016, 'train_samples_per_second': 9.777, 'train_steps_per_second': 1.222, 'total_flos': 841962936729600.0, 'train_loss': 0.08587084488681285, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'eval_loss': 3.208109046681784e-05,
 'eval_accuracy': 1.0,
 'eval_f1': 0.0,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 12.1668,
 'eval_samples_per_second': 32.876,
 'eval_steps_per_second': 2.055,
 'epoch': 2.0}

In [ ]:
trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'test_loss': 3.208287671441212e-05,
 'test_accuracy': 1.0,
 'test_f1': 0.0,
 'test_precision': 0.0,
 'test_recall': 0.0,
 'test_runtime': 31.6441,
 'test_samples_per_second': 31.601,
 'test_steps_per_second': 1.991,
 'epoch': 2.0}

Проверка на отзывах (взяла с imdb, отрицательный, положительный, отрицательный), вторая модель (выходы с пуллер слоя+cls токены):

In [27]:
reviews = ["This movie was disappointing to me because it had some really nice things going which made it promising for the first 66%. In the end however, it dragged to an over-the top disastrous climax which was really more gross than clever or insightful. I wish the end of the movie had been rewritten to perhaps have a confrontation with the drug makers rather than be all about her demise at a big public media event.", "I just watched one of the best films of the year. The Athens festival nights have kicked off, and at the opening ceremony, we were lucky enough to catch Anora. And let me tell you, this film absolutely slays.", "-Bad acting: a golden globe nomination and probably an Oscar-nomination too for the actress?!!!! FOR WHAT?!! "]
class_names = ['negative', 'positive']

In [28]:
for review in reviews:
    encoded_review = tokenizer.encode_plus(
        review,
        max_length=160,
        add_special_tokens=True,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model2(input_ids, attention_mask)
    logits = output['logits']
    prediction = torch.argmax(logits, dim=1)

    print(f'Review text: {review}')
    print(f'Sentiment  : {class_names[prediction]}')

Review text: This movie was disappointing to me because it had some really nice things going which made it promising for the first 66%. In the end however, it dragged to an over-the top disastrous climax which was really more gross than clever or insightful. I wish the end of the movie had been rewritten to perhaps have a confrontation with the drug makers rather than be all about her demise at a big public media event.
Sentiment  : negative
Review text: I just watched one of the best films of the year. The Athens festival nights have kicked off, and at the opening ceremony, we were lucky enough to catch Anora. And let me tell you, this film absolutely slays.
Sentiment  : negative
Review text: -Bad acting: a golden globe nomination and probably an Oscar-nomination too for the actress?!!!! FOR WHAT?!! 
Sentiment  : negative
